In [1]:
! pip install qiskit 
! pip install qiskit_aer


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from scipy.optimize import curve_fit

In [3]:
depths, sequences, shots = [1, 3, 5, 10, 15], 20, 1024
backend = AerSimulator()

In [4]:
def create_teleport_circuit():
    q = QuantumRegister(3, 'q')
    c = ClassicalRegister(2, 'c')
    qc = QuantumCircuit(q, c)
    
    qc.x(q[0])
    qc.barrier()
    qc.h(q[1])
    qc.cx(q[1], q[2])
    qc.cx(q[0], q[1])
    qc.h(q[0])
    qc.barrier()
    qc.measure(q[0], c[0])
    qc.measure(q[1], c[1])
    qc.barrier()
    with qc.if_test((c[1], 1)):
        qc.x(q[2])
    with qc.if_test((c[0], 1)):
        qc.z(q[2])
    
    return qc

In [5]:
survival_probs = []
for m in depths:
    seq_probs = []
    for _ in range(sequences):
        qc = QuantumCircuit(3, 3)
        for _ in range(m):
            teleport_qc = create_teleport_circuit()
            qc.compose(teleport_qc, inplace=True)
            qc.reset([0, 1])  
        qc.measure(2, 2)
        counts = backend.run(transpile(qc, backend), shots=shots).result().get_counts()
        seq_probs.append(counts.get('000', 0) / shots)   
    avg_prob = np.mean(seq_probs)
    survival_probs.append(avg_prob)
    print(f"Depth {m}: survival probability = {avg_prob:.4f}")

Depth 1: survival probability = 0.0000
Depth 3: survival probability = 0.0000
Depth 5: survival probability = 0.0000
Depth 10: survival probability = 0.2513
Depth 15: survival probability = 0.0000


In [6]:
try:
    popt, _ = curve_fit(lambda m, A, B, p: A * p**m + B, 
                        depths, survival_probs, 
                        p0=[0.8, 0.2, 0.95],  # Better initial guess
                        bounds=([0, 0, 0.8], [1, 1, 1.0]),  # Physical bounds
                        maxfev=5000)  # More iterations
    
    fidelity = 1 - (1 - popt[2]) * (2 - 1)/2
    print(f"\nProcess fidelity: {fidelity:.4f}")
    print(f"Fit parameters: A={popt[0]:.3f}, B={popt[1]:.3f}, p={popt[2]:.3f}")
    
except Exception as e:
    print(f"Fitting failed: {e}")
    print("Raw survival probabilities:", survival_probs)
    # Fallback: estimate from first two points
    if len(survival_probs) >= 2:
        p_est = survival_probs[1] / survival_probs[0] if survival_probs[0] > 0 else 0.95
        fidelity = 1 - (1 - p_est) * (2 - 1)/2
        print(f"Estimated fidelity: {fidelity:.4f}")


Process fidelity: 1.0000
Fit parameters: A=0.000, B=0.050, p=1.000
